In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as sps
import time



# Input data files are available in the read-only "./input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# Read Data

In [2]:
RM_train=pd.read_csv('./input/data_train.csv')
R_test=pd.read_csv('./input/data_target_users_test.csv')
URM=pd.read_csv('./input/data_train.csv')
ICM = pd.read_csv('./input/data_ICM_title_abstract.csv')

## URM all

In [3]:
URM_tuples = [tuple(x) for x in URM.to_numpy()]

userList, itemList, ratingList = zip(*URM_tuples)

userList = list(userList)
userList=np.array(userList,dtype=np.int64)
itemList = list(itemList)
itemList=np.array(itemList,dtype=np.int64)

ratingList = list(ratingList)                        #not needed
ratingList=np.array(ratingList,dtype=np.int64)       #not needed

URM_all = sps.coo_matrix((ratingList, (userList, itemList)))
URM_all = URM_all.tocsr()

In [4]:
userList_unique = list(set(userList))
itemList_unique = list(set(itemList))

numUsers = len(userList_unique)
numItems = len(itemList_unique)

numberInteractions= len(URM_tuples)
print ("Number of items\t {}, Number of users\t {}".format(numItems, numUsers))
print("Number of Intraction \t {}" .format(numberInteractions))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemList_unique), max(userList_unique)))
print ("Average interactions per user {:.2f}".format(numberInteractions/numUsers))
print ("Average interactions per item {:.2f}\n".format(numberInteractions/numItems))

print ("Sparsity {:.2f} %".format((1-float(numberInteractions)/(numItems*numUsers))*100))

Number of items	 24896, Number of users	 7947
Number of Intraction 	 113268
Max ID items	 25974, Max Id users	 7946

Average interactions per user 14.25
Average interactions per item 4.55

Sparsity 99.94 %


## ICM all

In [5]:

ICM_tuples = [tuple(x) for x in ICM.to_numpy()]
itemList_icm, featureList_icm, scoreList_icm = zip(*ICM_tuples)

itemList_icm = list(itemList_icm)
itemList_icm = np.array(itemList_icm,dtype=np.int64)

featureList_icm = list(featureList_icm)
featureList_icm = np.array(featureList_icm,dtype=np.int64)

scoreList_icm = list(scoreList_icm)
scoreList_icm = np.array(scoreList_icm,dtype=np.float64)

ICM_all = sps.coo_matrix((scoreList_icm, (itemList_icm, featureList_icm)))

ICM_all

<25975x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 490691 stored elements in COOrdinate format>

## Test users

In [6]:
userTestList = [x for x in R_test.to_numpy()]
userTestList = zip(*userTestList)
userTestList = [list(a) for a in userTestList][0]

In [7]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

# evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

## Let's implement SLIM BPR 

## Step 1 - Sampling

#### Create a mask of positive interactions. How to build it depends on the data

In [8]:
URM_mask = URM_train.copy()
URM_mask.data[URM_mask.data < 1] = 0

URM_mask.eliminate_zeros()
URM_mask

<7947x25975 sparse matrix of type '<class 'numpy.float64'>'
	with 90614 stored elements in Compressed Sparse Row format>

In [9]:
n_users = URM_mask.shape[0]
n_items = URM_mask.shape[1]


# Extract users having at least one interaction to choose from
eligibleUsers = []

for user_id in range(n_users):
    start_pos = URM_mask.indptr[user_id]
    end_pos = URM_mask.indptr[user_id+1]
    if len(URM_mask.indices[start_pos:end_pos]) > 0:
        eligibleUsers.append(user_id)
                

def sampleTriplet():
    
    # By randomly selecting a user in this way we could end up 
    # with a user with no interactions
    #user_id = np.random.randint(0, n_users)
    
    user_id = np.random.choice(eligibleUsers)
    
    # Get user seen items and choose one
    userSeenItems = URM_mask[user_id,:].indices
    pos_item_id = np.random.choice(userSeenItems)

    negItemSelected = False

    # It's faster to just try again then to build a mapping of the non-seen items
    while (not negItemSelected):
        neg_item_id = np.random.randint(0, n_items)

        if (neg_item_id not in userSeenItems):
            
            negItemSelected = True

    return user_id, pos_item_id, neg_item_id


In [10]:
for _ in range(10):
    print(sampleTriplet())

(3670, 20897, 19396)
(7538, 3971, 24663)
(2826, 13196, 25151)
(6431, 8245, 3943)
(2446, 17202, 13575)
(269, 13099, 16976)
(1053, 20624, 10945)
(6291, 9322, 1250)
(3805, 6964, 10140)
(7116, 1212, 7606)


## Step 2 - Computing prediction

#### The prediction depends on the model: SLIM, Matrix Factorization... 

### We have to initialize our model. In case of SLIM it works best to initialize S as zero, in case of MF you cannot because of how the gradient is computed and you have to initialize at random. Here we initialize SLIM at random just so that we have some numbers to show

In [11]:
similarity_matrix = np.random.random((n_items,n_items))
similarity_matrix[np.arange(n_items),np.arange(n_items)] = 0

In [12]:
user_id, positive_item_id, negative_item_id = sampleTriplet()

In [13]:
positive_item_id

1112

In [14]:
negative_item_id

11120

In [15]:
userSeenItems = URM_mask[user_id,:].indices
userSeenItems

array([ 1112,  4825,  9358, 19123, 23109, 24427, 25770], dtype=int32)

In [16]:
x_i = similarity_matrix[positive_item_id, userSeenItems].sum()
x_j = similarity_matrix[negative_item_id, userSeenItems].sum()

print("x_i is {:.2f}, x_j is {:.2f}".format(x_i, x_j))

x_i is 3.02, x_j is 4.34


## Step 3 - Computing gradient

#### The gradient depends on the objective function: RMSE, BPR... 

In [17]:
x_ij = x_i - x_j
x_ij

-1.3227909338162291

In [18]:
#### The original BPR paper uses the logarithm of the sigmoid of x_ij, whose derivative is the following

In [19]:
gradient = 1 / (1 + np.exp(x_ij))
gradient

0.7896456704188543

## Step 4 - Update model

#### How to update depends on the model itself, here we have just one paramether, the similarity matrix, so we perform just one update. In matrix factorization we have two.

#### We need a learning rate, which influences how fast the model will change. Small ones lead to slower convergence but often higher results

In [20]:
learning_rate = 1e-3

similarity_matrix[positive_item_id, userSeenItems] += learning_rate * gradient
similarity_matrix[positive_item_id, positive_item_id] = 0

similarity_matrix[negative_item_id, userSeenItems] -= learning_rate * gradient
similarity_matrix[negative_item_id, negative_item_id] = 0

#### Usually there is no relevant change in the scores over a single iteration

In [21]:
x_i = similarity_matrix[positive_item_id, userSeenItems].sum()
x_j = similarity_matrix[negative_item_id, userSeenItems].sum()

print("x_i is {:.2f}, x_j is {:.2f}".format(x_i, x_j))

x_i is 3.03, x_j is 4.34


## Step 5 - Write the iterative epochs

In [22]:
import time

def epochIteration():

    # Get number of available interactions
    numPositiveIteractions = int(URM_mask.nnz*0.01)

    start_time_epoch = time.time()
    start_time_batch = time.time()

    # Uniform user sampling without replacement
    for num_sample in range(numPositiveIteractions):

        # Sample
        user_id, positive_item_id, negative_item_id = sampleTriplet()
        
        userSeenItems = URM_mask[user_id,:].indices
        
        # Prediction
        x_i = similarity_matrix[positive_item_id, userSeenItems].sum()
        x_j = similarity_matrix[negative_item_id, userSeenItems].sum()
        
        # Gradient
        x_ij = x_i - x_j

        gradient = 1 / (1 + np.exp(x_ij))
        
        # Update
        similarity_matrix[positive_item_id, userSeenItems] += learning_rate * gradient
        similarity_matrix[positive_item_id, positive_item_id] = 0

        similarity_matrix[negative_item_id, userSeenItems] -= learning_rate * gradient
        similarity_matrix[negative_item_id, negative_item_id] = 0
        

        if(time.time() - start_time_batch >= 30 or num_sample == numPositiveIteractions-1):
            print("Processed {} ( {:.2f}% ) in {:.2f} seconds. Sample per second: {:.0f}".format(
                num_sample,
                100.0* float(num_sample)/numPositiveIteractions,
                time.time() - start_time_batch,
                float(num_sample) / (time.time() - start_time_epoch)))


            start_time_batch = time.time()


In [23]:
epochIteration()

Processed 905 ( 99.89% ) in 1.89 seconds. Sample per second: 478


In [24]:
import scipy.sparse as sps


def similarityMatrixTopK(item_weights, forceSparseOutput = True, k=100, verbose = False, inplace=True):
    """
    The function selects the TopK most similar elements, column-wise

    :param item_weights:
    :param forceSparseOutput:
    :param k:
    :param verbose:
    :param inplace: Default True, WARNING matrix will be modified
    :return:
    """

    assert (item_weights.shape[0] == item_weights.shape[1]), "selectTopK: ItemWeights is not a square matrix"

    start_time = time.time()

    if verbose:
        print("Generating topK matrix")

    nitems = item_weights.shape[1]
    k = min(k, nitems)

    # for each column, keep only the top-k scored items
    sparse_weights = not isinstance(item_weights, np.ndarray)

    if not sparse_weights:

        idx_sorted = np.argsort(item_weights, axis=0)  # sort data inside each column

        if inplace:
            W = item_weights
        else:
            W = item_weights.copy()

        # index of the items that don't belong to the top-k similar items of each column
        not_top_k = idx_sorted[:-k, :]
        # use numpy fancy indexing to zero-out the values in sim without using a for loop
        W[not_top_k, np.arange(nitems)] = 0.0

        if forceSparseOutput:
            W_sparse = sps.csr_matrix(W, shape=(nitems, nitems))

            if verbose:
                print("Sparse TopK matrix generated in {:.2f} seconds".format(time.time() - start_time))

            return W_sparse

        if verbose:
            print("Dense TopK matrix generated in {:.2f} seconds".format(time.time()-start_time))

        return W

    else:
        # iterate over each column and keep only the top-k similar items
        data, rows_indices, cols_indptr = [], [], []

        item_weights = check_matrix(item_weights, format='csc', dtype=np.float32)

        for item_idx in range(nitems):

            cols_indptr.append(len(data))

            start_position = item_weights.indptr[item_idx]
            end_position = item_weights.indptr[item_idx+1]

            column_data = item_weights.data[start_position:end_position]
            column_row_index = item_weights.indices[start_position:end_position]

            non_zero_data = column_data!=0

            idx_sorted = np.argsort(column_data[non_zero_data])  # sort by column
            top_k_idx = idx_sorted[-k:]

            data.extend(column_data[non_zero_data][top_k_idx])
            rows_indices.extend(column_row_index[non_zero_data][top_k_idx])


        cols_indptr.append(len(data))

        # During testing CSR is faster
        W_sparse = sps.csc_matrix((data, rows_indices, cols_indptr), shape=(nitems, nitems), dtype=np.float32)
        W_sparse = W_sparse.tocsr()

        if verbose:
            print("Sparse TopK matrix generated in {:.2f} seconds".format(time.time() - start_time))

        return W_sparse



In [25]:
class SLIM_BPR_Recommender(object):
    """ SLIM_BPR recommender with cosine similarity and no shrinkage"""

    def __init__(self, URM):
        self.URM = URM
        
        self.similarity_matrix = np.zeros((n_items,n_items))
        
        self.URM_mask = self.URM.copy()
        self.URM_mask.data[self.URM_mask.data < 1] = 0
        self.URM_mask.eliminate_zeros()
        
        self.n_users = URM_mask.shape[0]
        self.n_items = URM_mask.shape[1]


        # Extract users having at least one interaction to choose from
        self.eligibleUsers = []

        for user_id in range(n_users):

            start_pos = self.URM_mask.indptr[user_id]
            end_pos = self.URM_mask.indptr[user_id+1]

            if len(self.URM_mask.indices[start_pos:end_pos]) > 0:
                self.eligibleUsers.append(user_id)



    def sampleTriplet(self):

        # By randomly selecting a user in this way we could end up 
        # with a user with no interactions
        #user_id = np.random.randint(0, n_users)

        user_id = np.random.choice(self.eligibleUsers)

        # Get user seen items and choose one
        userSeenItems = URM_mask[user_id,:].indices
        pos_item_id = np.random.choice(userSeenItems)

        negItemSelected = False

        # It's faster to just try again then to build a mapping of the non-seen items
        while (not negItemSelected):
            neg_item_id = np.random.randint(0, n_items)

            if (neg_item_id not in userSeenItems):

                negItemSelected = True

        return user_id, pos_item_id, neg_item_id


        
    def epochIteration(self):

        # Get number of available interactions
        numPositiveIteractions = int(self.URM_mask.nnz*0.01)

        start_time_epoch = time.time()
        start_time_batch = time.time()

        # Uniform user sampling without replacement
        for num_sample in range(numPositiveIteractions):

            # Sample
            user_id, positive_item_id, negative_item_id = self.sampleTriplet()

            userSeenItems = self.URM_mask[user_id,:].indices

            # Prediction
            x_i = self.similarity_matrix[positive_item_id, userSeenItems].sum()
            x_j = self.similarity_matrix[negative_item_id, userSeenItems].sum()

            # Gradient
            x_ij = x_i - x_j

            gradient = 1 / (1 + np.exp(x_ij))

            # Update
            self.similarity_matrix[positive_item_id, userSeenItems] += learning_rate * gradient
            self.similarity_matrix[positive_item_id, positive_item_id] = 0

            self.similarity_matrix[negative_item_id, userSeenItems] -= learning_rate * gradient
            self.similarity_matrix[negative_item_id, negative_item_id] = 0


            if(time.time() - start_time_batch >= 30 or num_sample == numPositiveIteractions-1):
                print("Processed {} ( {:.2f}% ) in {:.2f} seconds. Sample per second: {:.0f}".format(
                    num_sample,
                    100.0* float(num_sample)/numPositiveIteractions,
                    time.time() - start_time_batch,
                    float(num_sample) / (time.time() - start_time_epoch)))

                start_time_batch = time.time()

                
    def fit(self, learning_rate = 0.01, epochs = 10):
 
        self.learning_rate = learning_rate
        self.epochs = epochs

        for numEpoch in range(self.epochs):
            self.epochIteration()
            
        self.similarity_matrix = self.similarity_matrix.T
        
        self.similarity_matrix = similarityMatrixTopK(self.similarity_matrix, k=100)
        
        
        
    def recommend(self, user_id, at=None, exclude_seen=True):
        # compute the scores using the dot product
        user_profile = self.URM[user_id]
        scores = user_profile.dot(self.similarity_matrix).toarray().ravel()

        if exclude_seen:
            scores = self.filter_seen(user_id, scores)

        # rank items
        ranking = scores.argsort()[::-1]
            
        return ranking[:at]
    
    
    def filter_seen(self, user_id, scores):

        start_pos = self.URM.indptr[user_id]
        end_pos = self.URM.indptr[user_id+1]

        user_profile = self.URM.indices[start_pos:end_pos]
        
        scores[user_profile] = -np.inf

        return scores  

In [26]:
recommender = SLIM_BPR_Recommender(URM_train)
recommender.fit(epochs=1)

Processed 905 ( 99.89% ) in 0.91 seconds. Sample per second: 993


In [27]:
from Notebooks_utils.evaluation_function import evaluate_algorithm

evaluate_algorithm(URM_test, recommender)

Evaluated user 0 of 7947
Recommender performance is: Precision = 0.0040, Recall = 0.0065, MAP = 0.0049


{'precision': 0.003955500618046964,
 'recall': 0.0064557170134016606,
 'MAP': 0.004903614103243273}

In [28]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython



recommender = SLIM_BPR_Cython(URM_train, recompile_cython=False)

recommender.fit(epochs=10, batch_size=1, sgd_mode='sgd', learning_rate=1e-4, positive_threshold_BPR=1)

SLIM_BPR_Recommender: URM Detected 77 (0.97 %) cold users.
SLIM_BPR_Recommender: URM Detected 2453 (9.44 %) cold items.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 7947 ( 100.00% ) in 0.50 seconds. BPR loss is 1.97E-08. Sample per second: 15885
SLIM_BPR_Recommender: Epoch 1 of 10. Elapsed time 0.02 sec
Processed 7947 ( 100.00% ) in 0.52 seconds. BPR loss is 6.44E-08. Sample per second: 15307
SLIM_BPR_Recommender: Epoch 2 of 10. Elapsed time 0.03 sec
Processed 7947 ( 100.00% ) in 0.54 seconds. BPR loss is 1.46E-07. Sample per second: 14699
SLIM_BPR_Recommender: Epoch 3 of 10. Elapsed time 0.06 sec
Processed 7947 ( 100.00% ) in 0.56 seconds. BPR loss is 2.57E-07. Sample per second: 14176
SLIM_BPR_Recommender: Epoch 4 of 10. Elapsed time 0.08 sec
Processed 7947 ( 100.00% ) in 0.58 seconds. BPR loss is 4.23

In [29]:
from Base.Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[5])


evaluator_validation.evaluateRecommender(recommender)

EvaluatorHoldout: Processed 5663 ( 100.00% ) in 3.21 sec. Users per second: 1764


({5: {'ROC_AUC': 0.09623874271587492,
   'PRECISION': 0.03775384072046581,
   'PRECISION_RECALL_MIN_DEN': 0.07308552592854156,
   'RECALL': 0.06499789142705964,
   'MAP': 0.043094698530421534,
   'MRR': 0.09212431573370977,
   'NDCG': 0.05607747518607622,
   'F1': 0.04776406175966225,
   'HIT_RATE': 0.18876920360233093,
   'ARHR': 0.10105067985166864,
   'NOVELTY': 0.002399682062707185,
   'AVERAGE_POPULARITY': 0.12868208471976456,
   'DIVERSITY_MEAN_INTER_LIST': 0.9861297372815402,
   'DIVERSITY_HERFINDAHL': 0.9971911203421536,
   'COVERAGE_ITEM': 0.2534744947064485,
   'COVERAGE_ITEM_CORRECT': 0.020134744947064485,
   'COVERAGE_USER': 0.712595948156537,
   'COVERAGE_USER_CORRECT': 0.11337611677362527,
   'DIVERSITY_GINI': 0.08680949790461166,
   'SHANNON_ENTROPY': 10.734623959280022}},
 'CUTOFF: 5 - ROC_AUC: 0.0962387, PRECISION: 0.0377538, PRECISION_RECALL_MIN_DEN: 0.0730855, RECALL: 0.0649979, MAP: 0.0430947, MRR: 0.0921243, NDCG: 0.0560775, F1: 0.0477641, HIT_RATE: 0.1887692, ARHR

In [30]:
recommender = SLIM_BPR_Cython(URM_all, recompile_cython=False)

recommender.fit(epochs=10, batch_size=1, sgd_mode='sgd', learning_rate=1e-4, positive_threshold_BPR=1)

SLIM_BPR_Recommender: URM Detected 1079 (4.15 %) cold items.
Deallocating Cython objects
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 7947 ( 100.00% ) in 0.45 seconds. BPR loss is 3.11E-08. Sample per second: 17607
SLIM_BPR_Recommender: Epoch 1 of 10. Elapsed time 0.02 sec
Processed 7947 ( 100.00% ) in 0.47 seconds. BPR loss is 1.28E-07. Sample per second: 16733
SLIM_BPR_Recommender: Epoch 2 of 10. Elapsed time 0.04 sec
Processed 7947 ( 100.00% ) in 0.50 seconds. BPR loss is 2.12E-07. Sample per second: 16004
SLIM_BPR_Recommender: Epoch 3 of 10. Elapsed time 0.06 sec
Processed 7947 ( 100.00% ) in 0.52 seconds. BPR loss is 4.46E-07. Sample per second: 15282
SLIM_BPR_Recommender: Epoch 4 of 10. Elapsed time 0.09 sec
Processed 7947 ( 100.00% ) in 0.54 seconds. BPR loss is 6.84E-07. Sample per second: 14634


In [31]:
recoms = recommender.recommend(userTestList, cutoff=10)

In [32]:
recomList = []
for i in range(len(recoms)):
    recomList.append(' '.join(str(e) for e in recoms[i]))
# print(recoms)

res = {"user_id": userTestList, "item_list": recomList}
result = pd.DataFrame(res, columns= ['user_id', 'item_list'])

result.to_csv ('outputs/slim_brp1.csv', index = False, header=True)

## SLIM ElasticNet

In [33]:
import numpy as np
import scipy.sparse as sps

from sklearn.linear_model import ElasticNet

import time, sys





class SLIMElasticNetRecommender(object):
    """
    Train a Sparse Linear Methods (SLIM) item similarity model.
    NOTE: ElasticNet solver is parallel, a single intance of SLIM_ElasticNet will
          make use of half the cores available

    See:
        Efficient Top-N Recommendation by Linear Regression,
        M. Levy and K. Jack, LSRS workshop at RecSys 2013.
        https://www.slideshare.net/MarkLevy/efficient-slides

        SLIM: Sparse linear methods for top-n recommender systems,
        X. Ning and G. Karypis, ICDM 2011.
        http://glaros.dtc.umn.edu/gkhome/fetch/papers/SLIM2011icdm.pdf
    """

    RECOMMENDER_NAME = "SLIMElasticNetRecommender"

    def __init__(self, URM_train):

        super(SLIMElasticNetRecommender, self).__init__()

        self.URM_train = URM_train


    def fit(self, l1_penalty=0.1, l2_penalty=0.1, positive_only=True, topK = 100):

        self.l1_penalty = l1_penalty
        self.l2_penalty = l2_penalty
        self.positive_only = positive_only
        self.topK = topK

        if self.l1_penalty + self.l2_penalty != 0:
            self.l1_ratio = self.l1_penalty / (self.l1_penalty + self.l2_penalty)
        else:
            print("SLIM_ElasticNet: l1_penalty+l2_penalty cannot be equal to zero, setting the ratio l1/(l1+l2) to 1.0")
            self.l1_ratio = 1.0



        # initialize the ElasticNet model
        self.model = ElasticNet(alpha=1.0,
                                l1_ratio=self.l1_ratio,
                                positive=self.positive_only,
                                fit_intercept=False,
                                copy_X=False,
                                precompute=True,
                                selection='random',
                                max_iter=100,
                                tol=1e-4)


        URM_train = sps.csc_matrix(self.URM_train)

        n_items = URM_train.shape[1]


        # Use array as it reduces memory requirements compared to lists
        dataBlock = 10000000

        rows = np.zeros(dataBlock, dtype=np.int32)
        cols = np.zeros(dataBlock, dtype=np.int32)
        values = np.zeros(dataBlock, dtype=np.float32)

        numCells = 0


        start_time = time.time()
        start_time_printBatch = start_time

        # fit each item's factors sequentially (not in parallel)
        for currentItem in range(n_items):

            # get the target column
            y = URM_train[:, currentItem].toarray()

            # set the j-th column of X to zero
            start_pos = URM_train.indptr[currentItem]
            end_pos = URM_train.indptr[currentItem + 1]

            current_item_data_backup = URM_train.data[start_pos: end_pos].copy()
            URM_train.data[start_pos: end_pos] = 0.0



            # fit one ElasticNet model per column
            self.model.fit(URM_train, y)

            # self.model.coef_ contains the coefficient of the ElasticNet model
            # let's keep only the non-zero values

            # Select topK values
            # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
            # - Partition the data to extract the set of relevant items
            # - Sort only the relevant items
            # - Get the original item index

            nonzero_model_coef_index = self.model.sparse_coef_.indices
            nonzero_model_coef_value = self.model.sparse_coef_.data


            local_topK = min(len(nonzero_model_coef_value)-1, self.topK)

            relevant_items_partition = (-nonzero_model_coef_value).argpartition(local_topK)[0:local_topK]
            relevant_items_partition_sorting = np.argsort(-nonzero_model_coef_value[relevant_items_partition])
            ranking = relevant_items_partition[relevant_items_partition_sorting]


            for index in range(len(ranking)):

                if numCells == len(rows):
                    rows = np.concatenate((rows, np.zeros(dataBlock, dtype=np.int32)))
                    cols = np.concatenate((cols, np.zeros(dataBlock, dtype=np.int32)))
                    values = np.concatenate((values, np.zeros(dataBlock, dtype=np.float32)))


                rows[numCells] = nonzero_model_coef_index[ranking[index]]
                cols[numCells] = currentItem
                values[numCells] = nonzero_model_coef_value[ranking[index]]

                numCells += 1


            # finally, replace the original values of the j-th column
            URM_train.data[start_pos:end_pos] = current_item_data_backup


            if time.time() - start_time_printBatch > 300 or currentItem == n_items-1:
                print("Processed {} ( {:.2f}% ) in {:.2f} minutes. Items per second: {:.0f}".format(
                                  currentItem+1,
                                  100.0* float(currentItem+1)/n_items,
                                  (time.time()-start_time)/60,
                                  float(currentItem)/(time.time()-start_time)))
                sys.stdout.flush()
                sys.stderr.flush()

                start_time_printBatch = time.time()


        # generate the sparse weight matrix
        self.W_sparse = sps.csr_matrix((values[:numCells], (rows[:numCells], cols[:numCells])),
                                       shape=(n_items, n_items), dtype=np.float32)




        
    def recommend(self, user_id, at=None, exclude_seen=True):
        # compute the scores using the dot product
        user_profile = self.URM_train[user_id]
        scores = user_profile.dot(self.W_sparse).toarray().ravel()

        if exclude_seen:
            scores = self.filter_seen(user_id, scores)

        # rank items
        ranking = scores.argsort()[::-1]
            
        return ranking[:at]
    
    
    def filter_seen(self, user_id, scores):

        start_pos = self.URM_train.indptr[user_id]
        end_pos = self.URM_train.indptr[user_id+1]

        user_profile = self.URM_train.indices[start_pos:end_pos]
        
        scores[user_profile] = -np.inf

        return scores  

In [34]:
recommender = SLIMElasticNetRecommender(URM_train)

recommender.fit()


Deallocating Cython objects
Processed 16567 ( 63.78% ) in 5.00 minutes. Items per second: 55
Processed 25975 ( 100.00% ) in 7.86 minutes. Items per second: 55


In [35]:
evaluate_algorithm(URM_test, recommender)

Evaluated user 0 of 7947
Recommender performance is: Precision = 0.0002, Recall = 0.0001, MAP = 0.0001


{'precision': 0.0002119018188239449,
 'recall': 5.517695072137684e-05,
 'MAP': 8.358349520277827e-05}